## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
#Import necessary libraries to fetch the train and test data...
import os
from zipfile import ZipFile
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.layers import LeakyReLU

tf.set_random_seed(42)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/Colab Notebooks/Computer Vision/Project 2 - Dog Breed/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
with ZipFile(project_path+'test.zip','r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
with ZipFile(project_path+'sample_submission.csv.zip','r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
with ZipFile(project_path+'labels.csv.zip','r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
df_labels = pd.read_csv('/content/labels.csv')

In [9]:
df_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [10]:
print('The total count of Dogs: ', df_labels.shape[0])
print('The number of Dog Breeds: ', len(df_labels.breed.unique()))

The total count of Dogs:  10222
The number of Dog Breeds:  120


In [11]:
print('The count of Dogs in each Dog Breed:')
df_labels.groupby('breed').size()

The count of Dogs in each Dog Breed:


breed
affenpinscher                      80
afghan_hound                      116
african_hunting_dog                86
airedale                          107
american_staffordshire_terrier     74
                                 ... 
welsh_springer_spaniel             79
west_highland_white_terrier        81
whippet                            95
wire-haired_fox_terrier            82
yorkshire_terrier                  82
Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
#Get lable encoding for labels

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df_labels.breed)
df_labels['breedEncoded'] = le.transform(df_labels.breed)

In [13]:
df_labels.head()

,id,breed,breedEncoded
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,19
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,37
2,001cdf01b096e06d78e9e5112d419397,pekinese,85
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,15
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,49


In [14]:
labels = tf.keras.utils.to_categorical(df_labels.breedEncoded)
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
labels.shape

(10222, 120)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
X_train = []
ImgCnt = df_labels.shape[0]

for i in range(ImgCnt):
  try:
    dummy = cv2.imread('/content/train/'+df_labels.id[i]+'.jpg')
    dummy = cv2.resize(dummy,(128,128)) # resize to have all the images of same size
    X_train.append(dummy)
  except exception as e:
    print(e)

In [17]:
print('x_train Count: ', len(X_train))

x_train Count:  10222


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
#Convert the list to numpy array for easy manipulation...
X_train_arr = np.asarray(X_train)

In [0]:
X_train_std = X_train_arr/255

In [0]:
X_train_std = X_train_std.reshape(X_train_std.shape[0], 128, 128, 3).astype('float32')

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [21]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_std, labels, test_size=0.2, random_state=2)
print ("No. of images in train dataset: ", len(X_train_split))
print ("No. of images in Validation dataset: ", len(X_val_split))

No. of images in train dataset:  8177
No. of images in Validation dataset:  2045


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = pd.read_csv('/content/sample_submission.csv')['id']

In [23]:
test_img.head()

0    000621fb3cbb32d8935728e48679680e
1    00102ee9d8eb90812350685311fe5890
2    0012a730dfa437f5f3613fb75efcd4ce
3    001510bc8570bbeee98c8d80c8a95ec1
4    001a5f3114548acdefa3d4da05474c2e
Name: id, dtype: object

Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
ImgCnt = len(test_img)

for i in range(ImgCnt):
  try:
      img = cv2.imread('/content/test/' + test_img[i] + '.jpg')
      img_resize = cv2.resize(img,(128,128)) #resize to have all the images of same size
      x_test_feature.append(img_resize)
  except Exception as e:
      print(e)

In [0]:
#x_test_feature = []
#i = 0 # initialisation
#for f in tqdm(test_img.values): # f for format ,jpg
#    img = cv2.imread('./test/{}.jpg'.format(f), 0)
#    img_resize = cv2.resize(img, (img_rows, img_cols)) 
#    x_test_feature.append(img_resize)

Normalize the test data and convert it into 4 dimensions

In [0]:
#Convert the list to numpy array for easy manipulation...
X_test_arr = np.asarray(x_test_feature)

In [0]:
X_test_std = X_test_arr/255

In [0]:
X_test_std = X_test_std.reshape(X_test_std.shape[0], 128, 128, 3).astype('float32')

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [28]:
model = Sequential()
model.add(Conv2D(filters=32, input_shape = (128,128,3), kernel_size=5)) 
model.add(Conv2D(filters=64, kernel_size=3))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=256, kernel_initializer = 'he_normal', activation = 'relu'))
model.add(Dense(units = 120, activation = 'softmax'))

### Use batch_size = 128 and epochs = 10 and execute the model

In [29]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [30]:
history = model.fit(X_train_split, y_train_split,
                    epochs=10, 
                    validation_data=(X_val_split, y_val_split),
                    verbose = 1,
                    batch_size=128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 8177 samples, validate on 2045 samples
Epoch 1/10





8177/8177 [==============================] - 30s 4ms/step - loss: 15.8102 - acc: 0.0079 - val_loss: 16.0078 - val_acc: 0.0068
Epoch 2/10
8177/8177 [==============================] - 12s 1ms/step - loss: 15.9841 - acc: 0.0083 - val_loss: 16.0078 - val_acc: 0.0068
Epoch 3/10
8177/8177 [==============================] - 12s 1ms/step - loss: 15.9841 - acc: 0.0083 - val_loss: 16.0078 - val_acc: 0.0068
Epoch 4/10
8177/8177 [==============================] - 12s 1ms/step - loss: 15.9841 - acc: 0.0083 - val_loss: 16.0078 - val_acc: 0.0068
Epoch 5/10
8177/8177 [==============================] - 12s 1ms/step - loss: 15.9841 - acc: 0.0083 - val_loss: 16.0078 - val_acc: 0.0068
Epoch 6/10
8177/8177 [==============================] - 12s 1ms/step - loss: 15.9841 - acc: 0.0083 - val_loss: 16.0078 - val_acc: 0.0068
Epoch 7/10
8177/8177 [======

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
train_datagen.fit(X_train_split)

In [0]:
val_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
val_datagen.fit(X_val_split)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = train_datagen.flow(X_train_split, y_train_split, batch_size=9)
val_generator = val_datagen.flow(X_val_split, y_val_split, batch_size=9)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [35]:
model.fit_generator(train_generator,\
                    epochs=10, steps_per_epoch=300, \
                    verbose=1,validation_data=val_generator, validation_steps = 300)

Epoch 1/10
300/300 [==============================] - 36s 121ms/step - loss: 15.9569 - acc: 0.0100 - val_loss: 16.0044 - val_acc: 0.0071
Epoch 2/10
300/300 [==============================] - 36s 121ms/step - loss: 15.9509 - acc: 0.0104 - val_loss: 16.0104 - val_acc: 0.0067
Epoch 3/10
300/300 [==============================] - 36s 121ms/step - loss: 16.0524 - acc: 0.0041 - val_loss: 16.0044 - val_acc: 0.0071
Epoch 4/10
300/300 [==============================] - 36s 121ms/step - loss: 15.9330 - acc: 0.0115 - val_loss: 16.0041 - val_acc: 0.0071
Epoch 5/10
300/300 [==============================] - 36s 121ms/step - loss: 15.9748 - acc: 0.0089 - val_loss: 16.0163 - val_acc: 0.0063
Epoch 6/10
300/300 [==============================] - 36s 120ms/step - loss: 16.0405 - acc: 0.0048 - val_loss: 16.0104 - val_acc: 0.0067
Epoch 7/10
300/300 [==============================] - 36s 121ms/step - loss: 15.9748 - acc: 0.0089 - val_loss: 16.0101 - val_acc: 0.0067
Epoch 8/10
300/300 [=====================

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
# import keras.backend as k
# k.set_image_dim_ordering('th')

from keras import backend as K
K.tensorflow_backend.set_image_dim_ordering('th')

In [38]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [39]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 3, None, None)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, None, None)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, None, None)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, None, None)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, None, None)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, None, None)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, None, None)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
def gen_model():
   #Create your own input format
    input = Input(shape=(3, 128, 128),name = 'image_input')

    #Use the generated model 
    output_vgg16_conv = base_model(input)

    #Add the fully-connected layers    
    #x = Flatten()(output_vgg16_conv)
    x = Dense(units=1024, activation = 'relu')(output_vgg16_conv)
    x = Dense(units=256, activation = 'relu')(x)
    o = Dense(units = 120, activation = 'softmax')(x)
    model = Model(input=input, output=[o])
    return model

In [41]:
 model_Final = gen_model()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=[<tf.Tenso...)`
  del sys.path[0]


In [42]:
model_Final.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 3, 128, 128)       0         
_________________________________________________________________
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_4 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_5 (Dense)              (None, 120)               30840     
Total params: 15,533,240
Trainable params: 15,533,240
Non-trainable params: 0
_________________________________________________________________


### Make all the layers in the base_model (VGG16) to be non-trainable

In [43]:
for layer in model_Final.layers:
  print (layer.name)

image_input
vgg16
dense_3
dense_4
dense_5


In [44]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model_Final.layers:
  if('vgg16' in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

    
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model_Final.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

image_input
False
vgg16
False
dense_3
True
dense_4
True
dense_5
True


### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
model_Final.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [0]:
X_train_split_th = np.rollaxis(X_train_split, 3, 1)  
X_val_split_th =  np.rollaxis(X_val_split, 3, 1)

In [47]:
history = model_Final.fit(X_train_split_th, y_train_split,
                    epochs=10, 
                    validation_data=(X_val_split_th, y_val_split),
                    verbose = 1,
                    batch_size=128)

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8177/8177 [==============================] - 11s 1ms/step - loss: 4.7295 - acc: 0.0247 - val_loss: 4.5817 - val_acc: 0.0411
Epoch 2/10
8177/8177 [==============================] - 8s 1ms/step - loss: 4.2571 - acc: 0.0753 - val_loss: 4.0726 - val_acc: 0.0885
Epoch 3/10
8177/8177 [==============================] - 8s 1ms/step - loss: 3.7082 - acc: 0.1475 - val_loss: 3.7249 - val_acc: 0.1315
Epoch 4/10
8177/8177 [==============================] - 8s 1ms/step - loss: 3.3688 - acc: 0.1904 - val_loss: 3.4889 - val_acc: 0.1672
Epoch 5/10
8177/8177 [==============================] - 8s 1ms/step - loss: 3.1266 - acc: 0.2358 - val_loss: 3.3614 - val_acc: 0.1785
Epoch 6/10
8177/8177 [==============================] - 8s 1ms/step - loss: 2.9549 - acc: 0.2753 - val_loss: 3.3198 - val_acc: 0.1892
Epoch 7/10
8177/8177 [==============================] - 8s 1ms/step - loss: 2.8299 - acc: 0.2966 - val_loss: 3.2589 - val_acc: 0.2108
Epoch 8/10
81